# LLM - HW2 : Convolutionnal Classifiaction Notebook - **Alexandre NGAU** *(November 8, 2023)*

# Environment setting

In [1]:
# ========== installing packages ==========

%%capture
!pip install transformers datasets

In [2]:
# ========== importing libraries ==========

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import random
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm
from transformers import BertTokenizer

# ========== designating the device (GPU when available) ==========

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cpu


# Data loading

In [3]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


# Pre-processing / Tokenization

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [5]:
def preprocessing_fn(x, tokenizer):
    x["review_ids"] = tokenizer(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )["input_ids"]
    x["label"] = 0 if x["sentiment"] == "negative" else 1
    return x

In [6]:
n_samples = 5000  # the number of training example for test

# We first shuffle the data !
dataset = dataset.shuffle()

# Select 5000 samples
split_dataset = dataset.select(range(n_samples))

# Tokenize the dataset
tok_dataset = split_dataset.map(preprocessing_fn,
                                fn_kwargs={"tokenizer" : tokenizer})

# Remove useless columns
splitted_dataset = tok_dataset.select_columns(["review_ids", "label"])

# Split the train and validation
splitted_dataset = splitted_dataset.train_test_split(test_size=0.2)

# ========== splitting the train/validation dataset ==========

train_set = splitted_dataset["train"]
valid_set = splitted_dataset["test"]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# Wrapping in a DataLoader

In [7]:
class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        # `batch` is a list of dictionary with keys "review_ids" and "label".
        features = [{"input_ids": x["review_ids"]} for x in batch]
        features = self.tokenizer.pad(
            features, padding="max_length", max_length=256, return_tensors="pt"
        )
        label = torch.tensor([x["label"] for x in batch])[:, None]
        return {"review_ids": features["input_ids"], "label": label}

In [8]:
data_collator = DataCollator(tokenizer)

In [9]:
batch_size = 64

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)

Up until the last cell (included), everything was recovered from previous lab sessions.

# Convolution model

# `load_model` function

In [10]:
def load_model(Word2Vec_model_PATH, freeze=False):
    word2vec_ckpt = torch.load(Word2Vec_model_PATH, map_location=DEVICE)
    pretrained_embd = word2vec_ckpt['embedding_words.weight']
    num_embeddings, embedding_dim = pretrained_embd.size()
    embedding_layer = nn.Embedding.from_pretrained(pretrained_embd, freeze=freeze)
    return embedding_layer

# `Conv1DModel` Model - using pretrained Word2Vec embeddings

In [11]:
class Conv1DModel(nn.Module):

    def __init__(self, Word2Vec_model_PATH, embedding_dim, max_length=256, feature_size=100, kernel_size=3):
        super(Conv1DModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.feature_size = feature_size

        self.embedding = load_model(Word2Vec_model_PATH, freeze=False)  #embedding layer initialized with trained Word2Vec
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv1d(embedding_dim, feature_size, kernel_size, padding=padding)
        self.pool = nn.MaxPool1d(max_length)
        self.fc = nn.Linear(feature_size, 1)

        self.sigmoid = nn.Sigmoid()


    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        conv_out = self.conv(embedded.mT)
        pooled = self.pool(conv_out)
        logits = self.fc(pooled.mT)
        probabilities = self.sigmoid(logits)

        return probabilities

# `Conv1DModelRandom` Model - using randomly initialized embeddings

In [12]:
vocab_size = len(tokenizer.get_vocab())

class Conv1DModelRandom(nn.Module):

    def __init__(self, embedding_dim, max_length=256, feature_size=100, kernel_size=3):
        super(Conv1DModelRandom, self).__init__()
        self.embedding_dim = embedding_dim
        self.feature_size = feature_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim) #embedding layer randomly initialized
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv1d(embedding_dim, feature_size, kernel_size, padding=padding)
        self.pool = nn.MaxPool1d(max_length)
        self.fc = nn.Linear(feature_size, 1)

        self.sigmoid = nn.Sigmoid()


    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        conv_out = self.conv(embedded.mT)
        pooled = self.pool(conv_out)
        logits = self.fc(pooled.mT)
        probabilities = self.sigmoid(logits)

        return probabilities

In the cell below, the `Word2Vec_PATH` variable is set to the model checkpoint in the Google Colab environmnent once the training of the Word2Vec (the whole other notebook) was entirely executed.

You might be needing to modify the name of the file (the path) in order to match the one you get if you use a smaller sample size, batch size, and number of epochs, for instance.

In [15]:
Word2Vec_PATH = '/content/model_dim-150_radius-4_ratio-6-batch-64-epoch-20.ckpt'
embedding_dimension = 150 # same as the one used to train the Word2Vec model

# Create the model with the trained Word2Vec embeddings
model = Conv1DModel(Word2Vec_PATH, embedding_dim=embedding_dimension)
model = model.to(DEVICE)

# Create the model with randomly initialized embeddings
model_random = Conv1DModelRandom(embedding_dim=embedding_dimension)
model_random = model_random.to(DEVICE)

In [16]:
def validation(model, valid_dataloader):

  # Tracking variables
  valid_loss = 0
  valid_acc = 0
  total_size = 0
  criterion = nn.BCELoss()

  # Set the model in evaluation mode
  model.eval()

  # ========== Evaluation ==========

  with torch.no_grad():
      for batch in tqdm(valid_dataloader):

          # Pushing the batches to the computing device
          review_ids, labels= (
          batch["review_ids"].to(DEVICE),
          batch["label"].to(DEVICE),
          )

          preds = model(review_ids)

          # Calculating the loss
          valid_loss = criterion(preds.squeeze(1), labels.float())
          valid_loss += valid_loss.detach().cpu().item()

          # Calculating the accuracy
          acc = torch.round(preds.squeeze().float()) == labels.squeeze().float()
          valid_acc += acc.int().sum().item()
          total_size += acc.numel()

  return valid_loss / len(valid_dataloader), valid_acc / total_size

In [17]:
def training(model, train_dataloader, valid_dataloader, n_epochs, lr=5e-5):
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        eps=1e-08,
    )
    criterion = nn.BCELoss()

    list_val_acc = []
    list_train_acc = []
    list_train_loss = []
    list_val_loss = []

    for e in range(1,n_epochs+1) :

        # ========== Training ==========

        # Set model to training mode
        model.train()

        # Tracking variables
        train_loss = 0
        epoch_train_acc = 0
        total_size = 0

        for batch in tqdm(train_dataloader):
          # Pushing the batches to the computing device
            review_ids, labels = (
                batch["review_ids"].to(DEVICE),
                batch["label"].to(DEVICE),
                )

            # Forward pass
            optimizer.zero_grad()
            outputs = model(review_ids)

            # Calculating loss
            loss = criterion(outputs.squeeze(1), labels.float())
            train_loss += loss.detach().cpu().item()

            # Backward pass
            loss.backward()
            optimizer.step()

            # Calculating accuracy
            acc = torch.round(outputs.squeeze().float()) == labels.squeeze().float()
            epoch_train_acc += acc.int().sum().item()
            total_size += acc.numel()

        list_train_acc.append(epoch_train_acc / total_size)
        list_train_loss.append(train_loss / len(train_dataloader))

        # ========== Validation ==========

        l, a = validation(model, valid_dataloader)
        list_val_loss.append(l)
        list_val_acc.append(a)
        print(
            e,
            "\n\t - Train loss: {:.4f}".format(list_train_loss[-1]),
            "Train acc: {:.4f}".format(list_train_acc[-1]),
            "Val loss: {:.4f}".format(l),
            "Val acc:{:.4f}".format(a),
        )

# Model training

In [18]:
# Training the model with pretrained Word2Vec embeddings
training(model, train_dataloader, valid_dataloader, n_epochs=50, lr=5e-5)

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

1 
	 - Train loss: 0.6943 Train acc: 0.5050 Val loss: 0.0874 Val acc:0.5310


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

49 
	 - Train loss: 0.3476 Train acc: 0.9882 Val loss: 0.0724 Val acc:0.7770


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

50 
	 - Train loss: 0.3393 Train acc: 0.9885 Val loss: 0.0720 Val acc:0.7780


In [19]:
# Training the model with randomly initialized embeddings
training(model_random, train_dataloader, valid_dataloader, n_epochs=50, lr=5e-5)

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

1 
	 - Train loss: 0.7169 Train acc: 0.5078 Val loss: 0.0841 Val acc:0.5380


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

49 
	 - Train loss: 0.3064 Train acc: 0.9353 Val loss: 0.0587 Val acc:0.7580


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

50 
	 - Train loss: 0.3009 Train acc: 0.9373 Val loss: 0.0583 Val acc:0.7570


The performances of these two classifying models in training have been logged into a plot that can be found in the corresponding PDF report, along with its analysis.